# Adam Forestier
## Last Updated: September 19, 2023

This File covers the following:
- Introduction to Tensors
- Getting information from Tensors
- Manipulating Tensors
- Tensors & NumPy
- Using @tf.function (a way to speed up regular Python functions)
- Using GPUs with TensorFlow (or TPUs)

### Introduction to Tensors

In [1]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
print(tf.__version__)

2.13.0


In [3]:
# Create tensors with tf.constant() - immutable
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check num dimensions of tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [5]:
# Create a vector
vector = tf.constant([10, 10])
l = [12, 1]
vector1 = tf.constant(l)
vector, vector1

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([12,  1], dtype=int32)>)

In [6]:
vector.ndim

1

In [7]:
# Create a matrix
mat = tf.constant([[10, 7],
                   [7, 10]])
mat

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [8]:
mat.ndim

2

In [9]:
mat1 = tf.constant([[11.1, 22.23],
                    [9., 9.99],
                    [22.32, 33.29]], dtype=tf.float16)
mat1

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[11.1 , 22.23],
       [ 9.  ,  9.99],
       [22.31, 33.28]], dtype=float16)>

In [10]:
mat1.ndim

2

In [11]:
tensor = tf.constant([[[1, 2, 3],
                       [4,5,6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [12]:
tensor.ndim

3

* scalar - 0 dimensions; single number
* vector - 1 dimension; a number with direction (e.g. wind speed & direction)
* matrix - 2 dimensions a 2-dimensional array of numbers
* tensor - 3 dimensions a n-dimensional array of numbers; where n can be any number. A 0-d tensor is a scalar, a 1-d tensor is a vector, a 2-d vector is a matrix

In [13]:
# Creating tensors with tf.variable - a mutable tensor
mutable = tf.Variable([10, 10])
immutable = tf.constant([10, 10])
mutable, immutable

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10, 10], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>)

In [14]:
# Change element in mutable tensor
mutable[0].assign(7)
mutable

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([ 7, 10], dtype=int32)>

**NOTE** Rarely in practice will you need to decide to use tf.constant or tf.Variable to create tensors, as TensorFlow does this for you. However, if in doubt, use tf.constant & change it later if needed

In [15]:
'''
Creating Random Tensors

Random tensors are tensors of some arbitrary size which contain random numbers
'''

# Set global set
tf.random.set_seed(42)

# set seed to make reproducable, uses Uniform distribution - distribution that has constant probability
rand = tf.random.Generator.from_seed(42)

# Normal distribution - function that represents the distribution of many random variables as a symmetric bell-shaped curve
rand = rand.normal(shape=(3,2))  # random.normal - outputs random values from a normal distribution
rand2 = tf.random.Generator.from_seed(42)
rand2 = rand2.normal(shape=(3,2))

# Show the two tensors, as well as an equality matrix. The matrix will be all True, because the random seed is
# set to be identical for both variables
rand, rand2, rand == rand2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

###Shuffling Tensor Order - September 7, 2023
- Why Shuffle?
- Shuffle data so the inherent order doesn't affect learning

In [16]:
# Create matrix to shuffle
not_shuffled = tf.constant([[10, 7],
                          [3, 4],
                          [2, 5]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [17]:
'''
Pseudo-randomly shuffle
tf.random.shuffle() will randomly shuffle along its first dimension
seed parameter is just for local operation
If one wants have reproducable shuffles, global and local operation needs to be set for shuffling (I set it globally a few cells up)

From docs: "If both global & operation seed are set: both seeds are used in conjunction to determine order"

Why? Want to have reproducable results each time experiment is run
'''
tf.random.set_seed(42)
shuffled = tf.random.shuffle(value=not_shuffled, seed=42)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [18]:
# Exercise - create 5 random tensors and then shuffle
tf.random.set_seed(42)
rand_tensors = []
i = 0
while i < 5:
  rand_generator = tf.random.Generator.from_seed(42)
  rand_tensor = rand_generator.normal(shape=(3,2))
  rand_tensors.append(rand_tensor)
  i += 1

i = 0
for t in rand_tensors:
  rand_tensors[i] = tf.random.shuffle(value=t, seed=42)
  i += 1

### Other Ways to Create Tensors

In [19]:
# ones
o = tf.ones(shape=(10, 7)) # Built in with Tensorflow

**IMPORTANT** - the main difference between NumPy arrays & Tensorflow Tensors, is that tensors can be run on a GPU (much faster for numerical computing)

**KEY** - use capital for matrix tensor, use lowercase for vector tensor

In [20]:
# using np array
A = np.array([[1, 2, 3], [4, 5, 6]])
a = np.arange(1, 25, dtype=np.int32)

# Easy, just pass as argument!
X = tf.constant(A)
x = tf.constant(a)

# Change the shape!
X2 = tf.constant(a, shape=(2,3,4)) # 2 matrices of 3 rows & 4 columns
print(f'{X2} \n\n {x}')

[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[13 14 15 16]
  [17 18 19 20]
  [21 22 23 24]]] 

 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


### Attributes - Get Information from Tensors

Key Attributes
- Shape: Length of each of dimensions of tensor
- Rank: Number of tensor dimensions. Scalar = 0, vector = 1, matrix = 2, tensor = 3
- Axis or Dimension: Dimension of a tensor
- Size: Total number of items in the tensor

In [21]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros(shape=(2,3,4,5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [22]:
# Attributes
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [23]:
# Get various attributes of tensor
print(f"Datatype: {rank_4_tensor.dtype}")
print(f"Number of dimensions (rank): {rank_4_tensor.ndim}")
print(f"Shape: {rank_4_tensor.shape}")
print(f"Elements along 0 axis: {rank_4_tensor.shape[0]}")
print(f"Elements along last axis: {rank_4_tensor.shape[-1]}")
print(f"Total elements: {tf.size(rank_4_tensor).numpy()}") # Cleaner way to display size

Datatype: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape: (2, 3, 4, 5)
Elements along 0 axis: 2
Elements along last axis: 5
Total elements: 120


###Indexing Tensors

In [24]:
# Get first 2 elemnts of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [25]:
# Get the first dimension from each index except the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [26]:
# Create a rank 2 tensor
Mat = tf.constant([[10, 7],
                   [12, 8]])
Mat.shape, mat.ndim

(TensorShape([2, 2]), 2)

In [27]:
# get last item of each row of rank 2 tensor
# All rows, last item!
Mat[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 8], dtype=int32)>

In [28]:
# Add in extra dimension
'''
... means: on every previous axis.
    The more verbose equivalent would be tensor = mat[:, :, tf.newaxis]
'''
tensor = Mat[..., tf.newaxis]
tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[12],
        [ 8]]], dtype=int32)>

In [29]:
# Add in extra dimension option 2
tensor = tf.expand_dims(input=Mat, axis=-1)
tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[12],
        [ 8]]], dtype=int32)>

In [30]:
# expand 0 axis
tensor = tf.expand_dims(input=Mat, axis=0)
tensor

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [12,  8]]], dtype=int32)>

### Manipulating Tensors (Tensor Operations)
**Basic Operations**
`+`, `-`, `*`, `/`

In [31]:
# Add values
tensor = tf.constant([[10, 7],
                      [90, 79]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 20,  17],
       [100,  89]], dtype=int32)>

In [32]:
# Multiple
tensor * 7

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 70,  49],
       [630, 553]], dtype=int32)>

In [33]:
# Subtract
# Tensors can go into negatives
tensor - 21

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[-11, -14],
       [ 69,  58]], dtype=int32)>

In [34]:
# can use tensorflow built-in functions too
# IMPORTANT!!! ALWAYS USE BUILT IN LIBRARIES. MUCH FASTER
tf.multiply(tensor, 10), tf.add(tensor, 2), tf.subtract(tensor, 88), tf.divide(tensor, 9)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  70],
        [900, 790]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[12,  9],
        [92, 81]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[-78, -81],
        [  2,  -9]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=float64, numpy=
 array([[ 1.11111111,  0.77777778],
        [10.        ,  8.77777778]])>)

### Matrix Multiplication - "Dot Product"

In ML, matrix multiplication is of the most common tensor operations

*Notes to self*
- https://www.mathsisfun.com/algebra/matrix-multiplying.html
- **Matrix B must have the same number of rows as Matrix A has Columns** AKA Inner Dimensions must match
- The resulting matrix has the shape of the outer dimensions. AKA Matrix A's rows and Matrix B's columns
- Example of math in Green notebook.

In [35]:
# tf.linalg.matmul() multiplies Matrix A by Matrix B resulting in A * B
MatA = tf.constant([[10, 23],
                    [88, 45]])
MatB = tf.constant([[10, 10],
                    [1, 22]])
Prod = tf.linalg.matmul(a=MatA, b=MatB)
Prod

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 123,  606],
       [ 925, 1870]], dtype=int32)>

In [36]:
# Matrix multiplication with Python operator "@" - learned something new!
# NOTE: Still faster to use Tensorflow built-in libraries!
Prod = MatA @ MatB
Prod

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 123,  606],
       [ 925, 1870]], dtype=int32)>

In [37]:
# Two random tensors multiplied together
rg = tf.random.Generator.from_seed(42)
MatA = rg.normal(shape=(3, 3))
rg = tf.random.Generator.from_seed(101)
MatB = rg.normal(shape=(3, 2))
tf.linalg.matmul(a=MatA, b=MatB)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.06664589,  1.2771904 ],
       [ 1.3243756 ,  2.3106525 ],
       [ 0.886186  , -0.20474419]], dtype=float32)>

In [38]:
# Example of two tensors to multiply without the same inner dimensions. must resphape
A = tf.constant([[3, 2],
                    [44, 22],
                    [9, 2]])
B = tf.constant([[44, 22],
                 [89, 22],
                 [67,22]])
'''
currently cannot multiply. 3 x 2 and 3 x 2 matrix multiplication does not work
must reshape so that inner dimensions match
'''
B_r = tf.reshape(tensor=B, shape=(2,3))
'''
now we have 3 x 2 and 2 x 3. Inner dimensions align
Resulting matrix will be the size of the outer dimensions (3 x 3)
'''
A, B_r

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 3,  2],
        [44, 22],
        [ 9,  2]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[44, 22, 89],
        [22, 67, 22]], dtype=int32)>)

In [39]:
# Resulting dot product
tf.linalg.matmul(a=A, b=B_r)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 176,  200,  311],
       [2420, 2442, 4400],
       [ 440,  332,  845]], dtype=int32)>

Transpose vs Reshape
- **Transpose** - flips along axis
- **Reshapes** - Shuffles values to get into correct shape
- Lots of reshaping & transposing done to tensors
- Generally: You do not have to choose, Tensorflow does transposing or reshaping behind the scenes for you
- As a rule of thumb: if you need to choose for yourself, more often than not, you want to transpose

In [40]:
# Use transpose to turn A into 2, 3 shape
A_t = tf.transpose(a=A)

# Show difference between reshape & transpose
print(f'Original\n {A}\n\n Transpose\n{A_t}\n\n Reshape\n{tf.reshape(tensor=A, shape=(2,3))}')

Original
 [[ 3  2]
 [44 22]
 [ 9  2]]

 Transpose
[[ 3 44  9]
 [ 2 22  2]]

 Reshape
[[ 3  2 44]
 [22  9  2]]


In [41]:
# Show difference again
A = np.random.randint(low=1, high=10, size=(3,2))
B = np.random.randint(low=1, high=10, size=(3,2))
r1 = tf.linalg.matmul(a=tf.transpose(a=A), b=B)
r2 = tf.linalg.matmul(a=tf.reshape(tensor=A, shape=(2,3)), b=B)
print(f'Matrix A Transpose before dot\n {r1}\n\n Matrix A Reshape before dot\n{r2}')

Matrix A Transpose before dot
 [[49 48]
 [97 98]]

 Matrix A Reshape before dot
[[41 47]
 [89 82]]


### Change datatype of tensor

In [42]:
# Show datatypes
v32 = tf.constant([1.7, 3.2])
vint = tf.constant([1, 2])
v32.dtype, vint.dtype

(tf.float32, tf.int32)

In [43]:
'''
Change from default 32 bit precision to 16 bit precision using casting
Modern hardware accelerators can before 16 bit dtype operations faster; potentially up to 2x faster!!! Tensor would take up half of the memory
'''
v16 = tf.cast(x=v32, dtype=tf.float16)
v16, v32

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 3.2], dtype=float16)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 3.2], dtype=float32)>)

In [44]:
# Another - int to float
v32_1 = tf.cast(x=vint, dtype=tf.float32)
v32_1

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

### Tensor Aggregation

Condensing from multiple values down to smaller amount of values

In [45]:
# Create some tensors
A = tf.constant(np.random.randint(low=-11, high=20, size=(2, 3)))
B = tf.constant(np.random.randint(low=-12, high=200, size=(2, 4)))
C = tf.constant(np.random.randint(low=-122, high=20, size=(4, 2)))

In [46]:
# Get absolute values
A, tf.abs(x=A)

(<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[17, -3, 13],
        [10, 14, 11]])>,
 <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[17,  3, 13],
        [10, 14, 11]])>)

In [47]:
# Min
B, tf.math.reduce_min(input_tensor=B).numpy()

(<tf.Tensor: shape=(2, 4), dtype=int64, numpy=
 array([[95, 93, -3,  1],
        [62, 81, 56, 65]])>,
 -3)

In [48]:
# Max
C, tf.math.reduce_max(input_tensor=C).numpy()

(<tf.Tensor: shape=(4, 2), dtype=int64, numpy=
 array([[  10,    0],
        [  14,  -80],
        [ -89, -105],
        [ -66,  -18]])>,
 14)

In [49]:
# Mean
tf.math.reduce_mean(input_tensor=B).numpy()

56

In [50]:
# Sum
tf.math.reduce_sum(input_tensor=A).numpy()

62

In [51]:
denom = tf.size(B).numpy()
num = tf.math.reduce_sum(input_tensor=B)
print(f'{num}/{denom}')
(num/denom).numpy()

450/8


56.25

In [52]:
# Variance
v = tf.math.reduce_variance(input_tensor=tf.cast(x=A, dtype=tf.float16)).numpy()
v

40.56

In [53]:
# Standard deviation
opt1 = tf.math.reduce_std(input_tensor=tf.cast(x=A, dtype=tf.float16)).numpy() # Option a
opt2 = tf.math.sqrt(x=v).numpy() # Option B
opt1, opt2

(6.367, 6.367)

### Positional Min & Max

In [54]:
# New tensor
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [55]:
tf.random.set_seed(42)
F = tf.random.shuffle(value=F, seed=42)

In [56]:
# At what index does max value occur? argmax
big_i = tf.math.argmax(input=F).numpy() # Returns index w/ the largeset value across axes of a tensor
print(f'value at biggest index, {big_i} = {F[big_i]}')

value at biggest index, 28 = 0.967138409614563


In [57]:
# At what index does min value occur? argmin
lil_i = tf.math.argmin(input=F).numpy() # Returns index w/ the largeset value across axes of a tensor
print(f'value at smallest index, {lil_i} = {F[lil_i]}')

value at smallest index, 29 = 0.009463667869567871


### Squeezing a Tensor
Removing all 1-dimension axes

In [58]:
# Create tensor
T = tf.random.uniform(shape=(1,1,1,1,50))
T

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [59]:
# squeeze
T.shape

TensorShape([1, 1, 1, 1, 50])

In [60]:
# Reduce extra dimensions with squeeze!
T_squeezed = tf.squeeze(input=T)
T_squeezed

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [61]:
T_squeezed.shape

TensorShape([50])

### One-hot encoding tensors

Numerical Encoding for Categorical Features
- Creates column for each classification, with 0 for when it is not instance of that classification, 1 when it is instance of that classification

In [62]:
# Create list
l = [0,1,2,3]

# One hot encode
ohe = tf.one_hot(indices=l, depth=4)
ohe

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [63]:
# Specify custom values for one hot encoding
ohe = tf.one_hot(indices=l, depth=4, on_value=True, off_value=False)
ohe_silly = tf.one_hot(indices=l, depth=4, on_value='cheese', off_value='bread')
ohe, ohe_silly

(<tf.Tensor: shape=(4, 4), dtype=bool, numpy=
 array([[ True, False, False, False],
        [False,  True, False, False],
        [False, False,  True, False],
        [False, False, False,  True]])>,
 <tf.Tensor: shape=(4, 4), dtype=string, numpy=
 array([[b'cheese', b'bread', b'bread', b'bread'],
        [b'bread', b'cheese', b'bread', b'bread'],
        [b'bread', b'bread', b'cheese', b'bread'],
        [b'bread', b'bread', b'bread', b'cheese']], dtype=object)>)

### More math operations

In [64]:
# create tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [65]:
# Square - performs on each element in tensor
tf.math.square(x=H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [66]:
# square root - performs on each element in tensor
tf.math.sqrt(x=tf.cast(x=H, dtype=tf.float16))

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([1.   , 1.414, 1.732, 2.   , 2.236, 2.45 , 2.646, 2.828, 3.   ],
      dtype=float16)>

In [67]:
# Log - performs on each element in tensor
tf.math.log(x=tf.cast(x=H, dtype=tf.float16))

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([0.    , 0.6934, 1.099 , 1.387 , 1.609 , 1.792 , 1.946 , 2.08  ,
       2.197 ], dtype=float16)>

### Tensorflow & NumPy compatibility

Tensorflow interacts well with NumPy array
- This is great, because if there is some functionality we want to perform with tensors, that is not supported by tensorflow, we can convert the tensor to a NumPy array, perform the operation, and then turn it back to a tensor!

In [68]:
# Create tensor from np array
J = tf.constant(np.array([1.0, 32.0, 90.0]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 1., 32., 90.])>

In [69]:
# Convert tensor to np array
np.array(J), type(np.array(J))

(array([ 1., 32., 90.]), numpy.ndarray)

In [70]:
# Convert tensor J TO NP array option 2
J.numpy(), type(J.numpy())

(array([ 1., 32., 90.]), numpy.ndarray)

In [71]:
# Default types. NP defaults to 64-bit precision, TF defaults to 32-bit precision
# Key to note if there are datatype errors
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Making sure our tensor operations run really fast on GPUs
- Numerical processing is MUCH faster on GPUs and TPUs

In [72]:
# Find access to hardware
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

*Steps*
1. Click Runtime
2. Select "change runtime type" from the dropdown
3. Select hardware accelerator
4. Save
5. Wait for colab notebook to connect to accelerator
6. Rerun notebook/cells!

In [75]:
# Display type of GPU being used
!nvidia-smi

Tue Sep 19 20:27:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |    553MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible!** No more additional setup needed